In [ ]:
##Python venv: search_agent_poc

In [2]:
##This module will take the serialized dictionary got out of PDF Parsing ann try to extract
##Semantic Knowldge like identifying 
## 1.Important Objects/Entities
## 2.Deduplicate Entities
## 3.Extracting Relations
## 4.Extract the main Ideas/Topics around Each Page
## 5.Link the different topics via diffrent entities/Objects
## 6.Break down the document by pages instead of Chunks .
## 7.If a page does not fit a chunk then chunk them extract information and then deduplicate the information across
## the page.

#Next Steps:
## 5.Try to Seggregate the BigPDF on Sections.
## 8.Try To Find Common Objects or ideas that link these sections.
## 9.Try 

In [12]:
import pickle
import pandas as pd
#rag_test_input_path='/home/matrix4284/MY_GEN_AI_PROJECTS/RAG/GraphRAG/graphrag-local-ollama/ragtest/input/'+file_name
import os
# importing shutil module
import shutil

from langchain_community.chat_models import ChatOllama

from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

from sentence_transformers import SentenceTransformer
#embeddings = SentenceTransformer("all-MiniLM-L6-v2")

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

import json
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
import json
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import XMLOutputParser

In [13]:
model_name="llama3.1"
book_text=''
page_text=''
file_name='Kubernetes_in_action_text_only'
extension='.txt'
start_page_idx=13
end_page_index=480
pdf_enrichment_output_dir = './pdf_enriched_output/'
#full_filename=file_name+'_'+str(page_idx)+extension
#full_filename

In [14]:
#LLM Model for Prompt Tuning
llm = ChatOllama(base_url="http://192.168.50.100:11434",model=model_name,temperature=0.15)

#embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
model_kwargs = {'device': 'cpu'}

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2",model_kwargs = model_kwargs)

##Define Vectorstore
vectorstore = Chroma(embedding_function=embeddings, persist_directory="./chroma_kubernetes_in_action")

In [16]:
#### Initial Load data (deserialized data).The input data is the enitity extraction as below:
#with open(pdf_enrichment_output_dir+'pdf_enriched_content_dict_phase1_entity_extraction_478_final.pickle', 'rb') as handle:
#    document_dict_deserialized = pickle.load(handle)

####If error happens start the load with intermediate index
with open(pdf_enrichment_output_dir+'pdf_enriched_content_dict_phase2_relation_extraction_12.pickle', 'rb') as handle:
    document_dict_deserialized = pickle.load(handle)
    


In [17]:
document_dict_deserialized[3]

{'page': 92,
 'img_cnt': 0,
 'img_npy_lst': [],
 'text': '60\nCHAPTER 3\nPods: running containers in Kubernetes\nFor example, the main container in a pod could be a web server that serves files from\na certain file directory, while an additional container (a sidecar container) periodi-\ncally downloads content from an external source and stores it in the web server’s\ndirectory. In chapter 6 you’ll see that you need to use a Kubernetes Volume that you\nmount into both containers. \n Other examples of sidecar containers include log rotators and collectors, data pro-\ncessors, communication adapters, and others.\nDECIDING WHEN TO USE MULTIPLE CONTAINERS IN A POD\nTo recap how containers should be grouped into pods—when deciding whether to\nput two containers into a single pod or into two separate pods, you always need to ask\nyourself the following questions:\n\uf0a1Do they need to be run together or can they run on different hosts?\n\uf0a1Do they represent a single whole or are they ind

In [18]:
##Definitions of Individual Enrichment Modules######

# Function to reverse a string
def reverse(string):
    string = string[::-1]
    return string

def entity_collector_per_page(entity_lst):

    entities=[]
    
    for entity in entity_lst:
        print("Entity:")
        print(entity)
        entity_name=entity['entity']
        entities.append(entity_name)
    return list(set(entities))


def extract_relationship_per_page(page_text,entity_lst):

# Prompt
    prompt = PromptTemplate(
            template="""You are a computer technology expert who has mastery in Kubernetes,Docker\n
            Machine Learning,Generative AI,Natural Language Understanding and Computer Vision.\n
            You have to deeply study a document page related to a particular technology and extract all \n
            the possible relations between the list of entities provided as input along with the context/document.\n
            While extracting relations carry out the following steps as per the below rules for each : \n
            1.You must define each relation by means of a json which must have values for three keys source_entity,description,destination_entity.
                a.source_entity should have that entity who is the one performing some action that is the subject. \n
                b.destination_entity should have the entity who is the object on which the source entity carries out \n
                one more more actions.\n
                c.Description of the relation will contain a brief summary about what action was being carried by the \n
                source_entity on the destination_entity. \n\n
            2.Make Sure the extracted source_entity,destination_entity and Description are always \n
            within "" and never within''.\n
            3.After all the relations have been extracted collate them into a list of json.
            4.Make sure the json is terminated properly.
            Output should only contain the list of json and no other words or character or sentences. \n
            The out json list must be terminated properly.
            Here is the document page: \n\n {context} \n\n and the entities: \n\n {entities} \n\n""",
            input_variables=["context","entities"],
    )

    # Chain
    chain = prompt | llm | StrOutputParser()

    # Score
    #filtered_docs = []

    output = chain.invoke(
        {
                "context": page_text,"entities":entity_lst
        }
    )
    
    #print('Relationship Output from LLM"')
    #print(output)
    
    if '[' in output:
        json_output=output.split('[')[1]
        json_output='['+json_output
    else:    
        json_output='['+output
    
    #print('JSON OUTPUT:')
    #print(json_output)
    
    

    json_output=reverse(json_output)
    
    if ']' in json_output:
        json_output=json_output.split(']')[1]
        json_output=reverse(json_output)
        json_output= json_output + ']'
    else:
        json_output=reverse(json_output)
        json_output= json_output + ']'
        
    
    
    #page_output_json=json.loads(output)
    #return page_output_json
    return json_output

In [19]:
######This code enrcihes the PDF page by page and while doing so it fails after processing 20 to 50 pages####
######currently we are manually tackling this problem by making the pdf dict list into aglobal variable######
######After the llm fails we take the index of the last page done and save the pdf dict list uptil that ####
#####Then as of now i am manually restarting the by loading the saved pdf dict list as the source pdf and ###
#####Using the last done pdf page index +1 as the starting index.So (last Done page Index + 1) is the ######
###starting index###
####The maximum page has been given as 565 as that is the last page where the main content of the document###
####resides.In case of the book Kubernetes in action it is 565.##############################################
####If this is batch process then the last page can be manually seen and used as the variable else one can ####
#####also go for the pdf enrichment of all the pages of the book#############################################

###This manual thing needs to be automated by either langraph or by using agents framework##################

def enrich_page(page_idx):
    
    #print("Page Number")
    #print(page_idx)
    
    entities_jsonl=document_dict_deserialized[page_idx]['entities']
    
    entity_lst=entity_collector_per_page(entities_jsonl)
    
    
    ##Relationship Extraction Enrichment
    if len(entity_lst)>0:
        page_relationship_lst_dict=extract_relationship_per_page(page_text,entity_lst)
        
        print("Page Relationship List Dictionary")
        print(page_relationship_lst_dict)
    
        document_dict_deserialized[page_idx]['relationships']=json.loads(page_relationship_lst_dict)
        print("Done for page "+str(page_idx))
        return page_idx
    else:
        print("there is no entity extracted from this page:"+str(page_idx))
        return page_idx
    

In [20]:
document_dict_deserialized[160]

{'page': 249,
 'img_cnt': 0,
 'img_npy_lst': [],
 'text': '217\nUsing Secrets to pass sensitive data to containers\nNOTE\nYou’re creating a generic Secret, but you could also have created a tls\nSecret with the kubectl create secret tls command, as you did in chapter 5.\nThis would create the Secret with different entry names, though.\n7.5.4\nComparing ConfigMaps and Secrets\nSecrets and ConfigMaps have a pretty big difference. This is what drove Kubernetes\ndevelopers to create ConfigMaps after Kubernetes had already supported Secrets for a\nwhile. The following listing shows the YAML of the Secret you created.\n$ kubectl get secret fortune-https -o yaml\napiVersion: v1\ndata:\n  foo: YmFyCg==\n  https.cert: LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSURCekNDQ...\n  https.key: LS0tLS1CRUdJTiBSU0EgUFJJVkFURSBLRVktLS0tLQpNSUlFcE...\nkind: Secret\n...\nNow compare this to the YAML of the ConfigMap you created earlier, which is shown\nin the following listing.\n$ kubectl get configmap fortun

In [ ]:

for idx in range(start_page_idx,end_page_index):
   
        enrich_page(idx)
        
        
        
        
    

Entity:
{'entity': 'Pods', 'description': 'running containers in Kubernetes', 'category': 'application'}
Entity:
{'entity': 'labels', 'description': 'creation_method: manual and env: prod', 'category': 'process'}
Entity:
{'entity': 'spec', 'description': 'containers: image: luksa/kubia, name: kubia, ports: containerPort: 8080, protocol: TCP', 'category': 'application'}
Entity:
{'entity': 'kubectl', 'description': 'create command to create a pod', 'category': 'command'}
Entity:
{'entity': 'get', 'description': 'kubectl get pods command to list pods', 'category': 'command'}
Entity:
{'entity': '--show-labels', 'description': 'switch to show labels in kubectl get pods output', 'category': 'option'}
Entity:
{'entity': '-L', 'description': 'switch to specify labels to display in kubectl get pods output', 'category': 'option'}
Entity:
{'entity': 'creation_method', 'description': 'label attached to a pod', 'category': 'process'}
Entity:
{'entity': 'env', 'description': 'label attached to a pod

Page Relationship List Dictionary
[
    {
        "source_entity": "Kubernetes",
        "description": "manages",
        "destination_entity": "pods"
    },
    {
        "source_entity": "UI pod",
        "description": "runs on",
        "destination_entity": "worker nodes"
    },
    {
        "source_entity": "Account Service",
        "description": "uses",
        "destination_entity": "SSDs"
    },
    {
        "source_entity": "Kubernetes",
        "description": "utilizes",
        "destination_entity": "GPU acceleration"
    },
    {
        "source_entity": "UI pod",
        "description": "requires",
        "destination_entity": "CPU"
    },
    {
        "source_entity": "Account Service",
        "description": "relies on",
        "destination_entity": "selectors"
    },
    {
        "source_entity": "Kubernetes",
        "description": "labels",
        "destination_entity": "pods"
    },
    {
        "source_entity": "UI pod",
        "description": "uses",
     

Page Relationship List Dictionary
[
  {"source_entity": "kubectl", "description": "annotate", "destination_entity": "metadata"},
  {"source_entity": "kubectl", "description": "describe", "destination_entity": "ReplicationController"},
  {"source_entity": "kubectl", "description": "create", "destination_entity": "namespace"},
  {"source_entity": "annotations", "description": "store", "destination_entity": "metadata"},
  {"source_entity": "yaml", "description": "format", "destination_entity": "JSON"},
  {"source_entity": "labels", "description": "assign", "destination_entity": "ReplicationController"},
  {"source_entity": "kubernetes.io/created-by", "description": "set", "destination_entity": "metadata"},
  {"source_entity": "kind", "description": "specify", "destination_entity": "ReplicationController"},
  {"source_entity": "kubectl annotate", "description": "add", "destination_entity": "annotations"}
]
Done for page 19
Entity:
{'entity': 'namespaces', 'description': 'groups objects int

In [11]:
import pickle

# Store data (serialize in a pickle) upto page 102
with open('./pdf_enriched_output/pdf_enriched_content_dict_phase2_relation_extraction_12.pickle', 'wb') as handle:
    pickle.dump(document_dict_deserialized, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
document_dict_deserialized[155]

{'page': 244,
 'img_cnt': 0,
 'img_npy_lst': [],
 'text': '212\nCHAPTER 7\nConfigMaps and Secrets: configuring applications\nEDITING A CONFIGMAP\nLet’s see how you can change a ConfigMap and have the process running in the pod\nreload the files exposed in the configMap volume. You’ll modify the Nginx config file\nfrom your previous example and make Nginx use the new config without restarting\nthe pod. Try switching gzip compression off by editing the fortune-config Config-\nMap with kubectl edit:\n$ kubectl edit configmap fortune-config\nOnce your editor opens, change the gzip on line to gzip off, save the file, and then\nclose the editor. The ConfigMap is then updated, and soon afterward, the actual file\nin the volume is updated as well. You can confirm this by printing the contents of the\nfile with kubectl exec:\n$ kubectl exec fortune-configmap-volume -c web-server\n➥  cat /etc/nginx/conf.d/my-nginx-config.conf\nIf you don’t see the update yet, wait a while and try again. It takes

In [ ]:
document_dict_deserialized[155]

In [ ]:
####The main contents of Kubernets In Action book is contained from page 87 to 565####
####Thats why we will only take the sliced portion i.e. from page 87 to 565 of the pdf dictionary list#####

In [48]:
# Load data (deserialize)
with open('pdf_content_dict_page_final.pickle', 'rb') as handle:
    document_dict_deserialized = pickle.load(handle)

In [ ]:
# Store data (serialize in a pickle) upto page 102
with open('../pdf_enrichment/pdf_enriched_output/pdf_enriched_content_dict_stage1.pickle', 'wb') as handle:
    pickle.dump(document_dict_deserialized, handle, protocol=pickle.HIGHEST_PROTOCOL)